# Portfolio Optimization, CAPM & Black-Litterman

__Assignment / Goal:__ Optimze the Dow Jones!  


> __You want to invest in a (optimized) Portfolio consisting of the 30 Dow Jones Stocks.__ 

1. Get all 30 Dow Jones Constituents from the Web (e.g.  "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")

2. Get daily prices (last 3Y) and Market Cap from Yahoo Finance and prepare daily returns and the covariance matrix. Get prices/returns for the Market Portfolio (S&P 500 - "^GSPC") as well. 

3. Calculate beta (3Y monthly) and identify all cyclical (beta > 1) and defensive (beta < 1) stocks. 

4. Calculate CAPM returns for all stocks. For the risk-free Asset, use the 13 Week US Treasury Bill ("^IRX"). For the expected Market Return, analyze two different scenarios:
- Bull Market (10%)
- Bear Market (0%)

5. Use CAPM returns (see 4.) for Portfolio Optimization (maximize Sharpe Ratio). Optimization Bounds: All Stocks shall have a minimum weight of 1% and a maximum weight of 25%!

6. Calculate and interpret the weighted average portfolio beta in
- Bull Market scenario (10%)
- Bear Market scenario (0%)

7. Calculate implied stock returns with Reverse Optimization (assume 10% market return) and compare with CAPM returns.

8. Add your own views and run a Black-Litterman Optimization. 

---

# Getting started

In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format
plt.style.use("seaborn-v0_8")

In [5]:
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"

In [6]:
# Get all Doe Jones instruments
df = pd.read_html(url)[1]
df

,Company,Exchange,Symbol,Industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.41%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.02%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,5.48%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,2.84%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.36%
5,Caterpillar,NYSE,CAT,Construction and mining,1991-05-06,NaN,4.52%
6,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,3.50%
7,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,0.96%
8,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.22%
9,Disney,NYSE,DIS,Broadcasting and entertainment,1991-05-06,NaN,1.89%


In [7]:
# Choose onlt those columns 
df = df[["Company", "Symbol", "Industry"]].copy()
df

,Company,Symbol,Industry
0,3M,MMM,Conglomerate
1,American Express,AXP,Financial services
2,Amgen,AMGN,Biopharmaceutical
3,Apple,AAPL,Information technology
4,Boeing,BA,Aerospace and defense
5,Caterpillar,CAT,Construction and mining
6,Chevron,CVX,Petroleum industry
7,Cisco,CSCO,Information technology
8,Coca-Cola,KO,Drink industry
9,Disney,DIS,Broadcasting and entertainment


In [8]:
# Set the symbol as index of the DataFrame
df.set_index("Symbol", inplace = True)
df

,Company,Industry
Symbol,,
MMM,3M,Conglomerate
AXP,American Express,Financial services
AMGN,Amgen,Biopharmaceutical
AAPL,Apple,Information technology
BA,Boeing,Aerospace and defense
CAT,Caterpillar,Construction and mining
CVX,Chevron,Petroleum industry
CSCO,Cisco,Information technology
KO,Coca-Cola,Drink industry


In [9]:
# Save the symbol as a 1D lists
symbols = df.index.to_list()
symbols

['MMM',
 'AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'CRM',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WBA',
 'WMT']

In [10]:
# Index portfolio = ^GSPC = Doe Jones
index = "^GSPC"

In [11]:
# Append the Index portfolio to the symbols list
symbols.append(index)
symbols

['MMM',
 'AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CVX',
 'CSCO',
 'KO',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'MCD',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'CRM',
 'TRV',
 'UNH',
 'VZ',
 'V',
 'WBA',
 'WMT',
 '^GSPC']

In [12]:
# Download the historical data of all the symbols. 
prices = yf.download(symbols, period = "3y", interval = "1d")
prices

[*********************100%%**********************]  31 of 31 completed


Adj Close                                                         \
                AAPL   AMGN    AXP     BA    CAT    CRM  CSCO    CVX    DIS   
Date                                                                          
2020-11-11    117.42 218.97 106.85 182.15 158.93 254.18 35.94  72.47 137.82   
2020-11-12    117.14 215.74 106.83 176.72 158.62 249.42 35.34  70.89 135.52   
2020-11-13    117.19 217.41 110.57 187.11 161.04 249.51 37.84  72.97 138.36   
2020-11-16    118.21 216.77 114.11 202.40 162.42 249.89 38.88  78.18 144.67   
2020-11-17    117.32 212.90 112.01 210.05 161.22 256.17 38.28  77.62 144.50   
...              ...    ...    ...    ...    ...    ...   ...    ...    ...   
2023-11-06    178.99 272.83 152.99 192.95 238.28 207.42 53.29 147.00  84.02   
2023-11-07    181.58 271.11 152.98 191.41 234.92 211.84 53.13 144.41  84.59   
2023-11-08    182.65 273.26 153.01 191.83 234.82 211.47 53.00 142.40  84.50   
2023-11-09    182.17 264.06 152.37 193.33 234.15 210.01 52.00 141.94  90.34   
2023-11-10    186.40 267.31 154.38 196.65 239.68 213.63 52.59 142.95  88.27   

                  ...    Volume                                                \
             DOW  ...      MSFT      NKE       PG      TRV      UNH         V   
Date              ...                                                           
2020-11-11 45.16  ...  29440800  6058700  5554300   966300  2048200   5337200   
2020-11-12 44.14  ...  21593900  4780400  5631700  1289500  2907300   7302500   
2020-11-13 44.95  ...  18621100  3986100  6466800  1135000  2442100   6377500   
2020-11-16 46.64  ...  24953300  6116200  7976600  1163500  2790100  12083200   
2020-11-17 47.12  ...  24154100  7002900  6396800  1218000  2557600   7268800   
...          ...  ...       ...      ...      ...      ...      ...       ...   
2023-11-06 49.34  ...  23828300  5231900  6480800  1327700  2684900   5254100   
2023-11-07 48.53  ...  25833900  7745800  6003800  1022300  2604800   4219700   
2023-11-08 48.30  ...  26767800  8639700  4832600  1165400  1769200   5117500   
2023-11-09 48.36  ...  24847300  9154200  6581500   933400  1819600   4866600   
2023-11-10 49.48  ...  28042100  8897700  5387200   911600  2480800   4094400   

                                                      
                  VZ       WBA       WMT       ^GSPC  
Date                                                  
2020-11-11  12196900   7338300   6268600  4635560000  
2020-11-12  13293300   5858400   4188100  4909660000  
2020-11-13  10652800   6447500   6579400  4719580000  
2020-11-16  14289400   8055800  11520100  5295510000  
2020-11-17  11698200  25101600  14237200  4812180000  
...              ...       ...       ...         ...  
2023-11-06  15544000   7649000   4843000  3656340000  
2023-11-07  17654600   6667400   5007800  3791230000  
2023-11-08  16971500   8060300   5197300  3729510000  
2023-11-09  15685800   8286000   5026700  3900780000  
2023-11-10  12791200   8793700   4773300  3665080000  

[755 rows x 186 columns]

In [13]:
# For each instrument compute the return of the close price.
returns = prices["Close"].pct_change()
returns

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-11-12,-0.00,-0.01,-0.00,-0.03,-0.00,-0.02,-0.02,-0.02,-0.02,-0.02,...,-0.01,-0.01,0.00,-0.02,0.01,-0.02,-0.00,-0.02,0.00,-0.01
2020-11-13,0.00,0.00,0.04,0.06,0.02,0.00,0.07,0.03,0.02,0.02,...,0.00,0.01,0.01,0.02,0.00,0.01,0.00,0.05,0.02,0.01
2020-11-16,0.01,-0.00,0.03,0.08,0.01,0.00,0.03,0.07,0.05,0.04,...,0.00,0.01,-0.01,0.01,0.00,0.01,-0.00,0.03,0.01,0.01
2020-11-17,-0.01,-0.02,-0.02,0.04,-0.01,0.03,-0.02,-0.02,-0.00,0.01,...,-0.01,0.02,-0.00,-0.01,-0.01,-0.01,-0.00,-0.10,-0.02,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,0.01,0.01,0.00,-0.01,-0.01,-0.00,0.01,-0.00,-0.01,-0.01,...,0.01,0.00,0.01,0.01,0.00,-0.00,-0.01,-0.02,0.00,0.00
2023-11-07,0.01,-0.01,-0.00,-0.01,-0.01,0.02,-0.00,-0.02,0.01,-0.02,...,0.01,0.02,-0.00,0.00,0.01,0.01,0.01,-0.01,0.00,0.00
2023-11-08,0.01,0.01,0.00,0.00,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,...,0.01,0.00,-0.00,-0.01,-0.00,-0.00,-0.00,-0.03,-0.01,0.00


In [14]:
# For each instrument, compute the annualize covariance matrix (risk). 
# Its the covariance of EACH INSTRUMENT WITH THE INDEX PORTFOLIO! 
cov_matrix = returns[df.index].cov() * 252
cov_matrix

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DIS,...,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT
MMM,0.06,0.03,0.02,0.02,0.03,0.03,0.02,0.02,0.02,0.03,...,0.02,0.03,0.01,0.02,0.02,0.01,0.02,0.02,0.03,0.01
AXP,0.03,0.09,0.01,0.04,0.06,0.05,0.03,0.03,0.02,0.05,...,0.03,0.04,0.01,0.04,0.03,0.02,0.01,0.04,0.03,0.01
AMGN,0.02,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01
AAPL,0.02,0.04,0.01,0.08,0.04,0.02,0.02,0.03,0.02,0.04,...,0.06,0.05,0.02,0.06,0.01,0.02,0.01,0.04,0.03,0.02
BA,0.03,0.06,0.01,0.04,0.15,0.05,0.04,0.03,0.01,0.06,...,0.03,0.05,0.01,0.05,0.03,0.01,0.01,0.04,0.04,0.01
CAT,0.03,0.05,0.01,0.02,0.05,0.09,0.05,0.02,0.01,0.04,...,0.02,0.03,0.01,0.02,0.03,0.01,0.01,0.03,0.03,0.01
CVX,0.02,0.03,0.01,0.02,0.04,0.05,0.08,0.02,0.01,0.03,...,0.01,0.02,0.00,0.02,0.03,0.01,0.01,0.02,0.03,0.01
CSCO,0.02,0.03,0.01,0.03,0.03,0.02,0.02,0.05,0.02,0.03,...,0.03,0.03,0.02,0.03,0.02,0.02,0.01,0.02,0.02,0.01
KO,0.02,0.02,0.01,0.02,0.01,0.01,0.01,0.02,0.03,0.02,...,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.01
DIS,0.03,0.05,0.01,0.04,0.06,0.04,0.03,0.03,0.02,0.10,...,0.04,0.05,0.01,0.05,0.02,0.01,0.01,0.04,0.03,0.01


In [15]:
count = 0

# For each instriment, get its Market cap
for ticker in df.index:
    count += 1
    print(count, end = '\r')
    try:
        #info = yf.Ticker(ticker).get_info()
        #df.loc[ticker, "beta"] = info["beta"]
        fast_info = yf.Ticker(ticker).fast_info
        df.loc[ticker, "mcap"] = fast_info["market_cap"] # store the Market cap for each instrument.
    except Exception as e:
        print(ticker, e)

In [16]:
# Have a look - each instrument have its own Market cap
df

,Company,Industry,mcap
Symbol,,,
MMM,3M,Conglomerate,51260538679.09
AXP,American Express,Financial services,112503808568.25
AMGN,Amgen,Biopharmaceutical,143058425596.45
AAPL,Apple,Information technology,2899041777354.84
BA,Boeing,Aerospace and defense,118968728077.11
CAT,Caterpillar,Construction and mining,125533113286.48
CVX,Chevron,Petroleum industry,269853859026.24
CSCO,Cisco,Information technology,214183813631.37
KO,Coca-Cola,Drink industry,245223815940.00


---

# Cyclical Stocks vs. Defensive Stocks

In [17]:
# For each instrument, compute the monthly return of the close price
returns_m = prices.Close.resample("M").last().pct_change()
returns_m

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
2020-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-31,0.11,0.04,0.02,0.02,0.05,-0.09,0.04,-0.03,0.22,0.05,...,0.04,0.05,0.00,0.08,0.04,0.04,-0.03,0.05,-0.06,0.04
2021-01-31,-0.01,0.05,-0.04,-0.09,0.00,0.01,-0.00,0.01,-0.07,-0.06,...,0.04,-0.06,-0.08,-0.03,-0.05,-0.12,-0.07,0.26,-0.03,-0.01
2021-02-28,-0.08,-0.07,0.16,0.09,0.18,-0.04,0.01,0.17,0.12,0.14,...,0.00,0.01,-0.04,0.07,-0.00,0.10,0.01,-0.05,-0.08,0.03
2021-03-31,0.01,0.11,0.05,0.20,0.07,-0.02,0.15,0.05,-0.02,0.08,...,0.01,-0.01,0.10,0.03,0.12,-0.00,0.05,0.15,0.05,0.04
2021-04-30,0.08,-0.04,0.08,-0.08,-0.02,0.09,-0.02,-0.02,0.01,-0.02,...,0.07,-0.00,-0.01,0.03,0.07,0.10,-0.01,-0.03,0.03,0.05
2021-05-31,-0.05,-0.01,0.04,0.05,0.06,0.03,0.04,0.01,-0.04,0.09,...,-0.01,0.03,0.01,0.03,0.03,-0.03,-0.02,-0.01,0.02,0.01
2021-06-30,0.10,0.02,0.03,-0.03,-0.10,0.03,0.00,0.01,-0.02,-0.08,...,0.08,0.13,0.00,-0.06,-0.03,0.03,-0.01,-0.00,-0.01,0.02
2021-07-31,0.06,-0.01,0.03,-0.05,-0.05,-0.01,0.04,-0.03,0.00,-0.02,...,0.05,0.08,0.05,-0.01,0.03,0.05,-0.00,-0.10,0.01,0.02


In [18]:
# Compute the beta 
# covariance of each stock with ^GSPC / variance ^GSPC 
df["beta"] = returns_m.cov()["^GSPC"] / returns_m["^GSPC"].var() # 3Y monthly beta
df

,Company,Industry,mcap,beta
Symbol,,,,
MMM,3M,Conglomerate,51260538679.09,1.09
AXP,American Express,Financial services,112503808568.25,1.02
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59
AAPL,Apple,Information technology,2899041777354.84,1.28
BA,Boeing,Aerospace and defense,118968728077.11,1.24
CAT,Caterpillar,Construction and mining,125533113286.48,1.45
CVX,Chevron,Petroleum industry,269853859026.24,0.82
CSCO,Cisco,Information technology,214183813631.37,1.01
KO,Coca-Cola,Drink industry,245223815940.00,0.52


__Cyclical Stocks__

In [19]:
# Its cyclical stocks if the beta > 1. 
df.loc[df.beta > 1].sort_values(by = "beta", ascending = False)

,Company,Industry,mcap,beta
Symbol,,,,
CAT,Caterpillar,Construction and mining,125533113286.48,1.45
NKE,Nike,Clothing industry,161489872726.66,1.36
DIS,Disney,Broadcasting and entertainment,157208858372.02,1.33
AAPL,Apple,Information technology,2899041777354.84,1.28
GS,Goldman Sachs,Financial services,106152720304.69,1.26
BA,Boeing,Aerospace and defense,118968728077.11,1.24
CRM,Salesforce,Information technology,208503103522.75,1.18
MMM,3M,Conglomerate,51260538679.09,1.09
INTC,Intel,Semiconductor industry,163941789644.38,1.06


__Defensive Stocks__

In [20]:
# Its Defensive Stocks if beta < 1
df.loc[df.beta < 1].sort_values(by = "beta", ascending = True)

,Company,Industry,mcap,beta
Symbol,,,,
VZ,Verizon,Telecommunications industry,150128410579.19,0.28
JNJ,Johnson & Johnson,Pharmaceutical industry,354471998848.00,0.34
MRK,Merck,Pharmaceutical industry,256924295986.80,0.34
PG,Procter & Gamble,Fast-moving consumer goods,356856740489.11,0.41
IBM,IBM,Information technology,136072993704.79,0.44
KO,Coca-Cola,Drink industry,245223815940.00,0.52
TRV,Travelers,Insurance,38818696933.12,0.52
UNH,UnitedHealth Group,Managed health care,500578675243.38,0.55
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59


---

## CAPM Returns and Portfolio Optimization

In [21]:
# Plot the instuments. 
# so far we have thier Market Cap and beta.
df

,Company,Industry,mcap,beta
Symbol,,,,
MMM,3M,Conglomerate,51260538679.09,1.09
AXP,American Express,Financial services,112503808568.25,1.02
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59
AAPL,Apple,Information technology,2899041777354.84,1.28
BA,Boeing,Aerospace and defense,118968728077.11,1.24
CAT,Caterpillar,Construction and mining,125533113286.48,1.45
CVX,Chevron,Petroleum industry,269853859026.24,0.82
CSCO,Cisco,Information technology,214183813631.37,1.01
KO,Coca-Cola,Drink industry,245223815940.00,0.52


__Risk-free Rate__

In [22]:
# Load the Rist Free Return of the market. 
rfr = yf.download(tickers = "^IRX").Close.dropna().iloc[-1] / 100 #13 Week US Treasury Bill
rfr

[*********************100%%**********************]  1 of 1 completed


0.0525

__Market Return Forecast (bullish)__

In [23]:
rm = 0.10

__Market Return Forecast (bearish)__

In [24]:
# rm = 0.00

### __Compute the CAPM return for each instrument__

In [25]:
# Compute the CAPM
# Risk free return + expected return market - Risk free return * beta of each instrument    
df["CAPM_ret"] = rfr + (rm - rfr) * df.beta
df

,Company,Industry,mcap,beta,CAPM_ret
Symbol,,,,,
MMM,3M,Conglomerate,51260538679.09,1.09,0.10
AXP,American Express,Financial services,112503808568.25,1.02,0.10
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11
BA,Boeing,Aerospace and defense,118968728077.11,1.24,0.11
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12
CVX,Chevron,Petroleum industry,269853859026.24,0.82,0.09
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10
KO,Coca-Cola,Drink industry,245223815940.00,0.52,0.08


In [26]:
# Function for calculate the portfolio return with given set of wegiths
def port_ret(weights):
    # note: we use CAPM returns as the return price of the instrument. 
    # CAPM returns * weigths = portfolio return! 
    ret = df.CAPM_ret.dot(weights)
    return ret 

In [27]:
# Function that calculate the portfolio volatility (risk) with giveb set of wegiths 
def port_vol(weights):
    risk = np.sqrt(weights.dot(cov_matrix).dot(weights))
    return risk

In [28]:
# Functin that calculate the sharp ratio of a portfolio with given set of wegiths
def sharpe(weights): 
     # the ratio between portfolio return & portfolio volatility = sharp ratio
     # -> return / risk = sharp ratio !
     return -(port_ret(weights)-rfr) / port_vol(weights) 

In [29]:
# For optimization
import scipy.optimize as sco

In [30]:
# Number of asset
noa = len(df)
noa

30

In [31]:
# Initial weigths (starting point for the optimization)
eweigths = np.full(noa, 1/noa)
eweigths

array([0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333])

In [32]:
#constraint: weights must sum up to 1 -> sum of weights - 1 = 0
cons = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})

In [33]:
# bounds - each weigths must by between 1% to 25% 
bnds =  tuple((0.01, 0.25) for x in range(noa))
bnds

((0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25))

In [34]:
# Run the optimizer
opts = sco.minimize(sharpe, # function to minimize (find the optimal sharp ratio)
                    eweigths, # initial weights 
                    method = "SLSQP", # optimizer
                    bounds = bnds,  # bounds
                    constraints= cons) # constraints

In [35]:
# Grab the wegiths of the optimal portfolio
optimal_weights = opts["x"]
optimal_weights

array([0.07986982, 0.01      , 0.03334118, 0.11905413, 0.01      ,
       0.18095953, 0.01      , 0.05380355, 0.01      , 0.05087915,
       0.01      , 0.05401759, 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.10797293, 0.01      ,
       0.01      , 0.02992812, 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.09017402])

In [36]:
# store the optimal weigths as a new column in our data frame
df["CAPM_weights"] = pd.Series(data = optimal_weights, index = df.index)

In [37]:
# Show the instruments and their optimal weigths from top to bottom
df.sort_values(by = "CAPM_weights", ascending = False)

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights
Symbol,,,,,,
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12
MCD,McDonald's,Food industry,194239344662.63,0.69,0.09,0.11
WMT,Walmart,Retailing,447310352335.03,0.62,0.08,0.09
MMM,3M,Conglomerate,51260538679.09,1.09,0.10,0.08
GS,Goldman Sachs,Financial services,106152720304.69,1.26,0.11,0.05
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05
DIS,Disney,Broadcasting and entertainment,157208858372.02,1.33,0.12,0.05
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03


In [38]:
# Compute the weighted avara beta
weighted_average_beta = df.beta.mul(df.CAPM_weights).sum()
weighted_average_beta

# This is the beta value of our optimized portfolio! 

1.0376064192162309

- __Expected Bull Market: High Beta Portfolio__
- __Expected Bear Market: Low Beta Portfolio__

---

# Reverse Optimization (Black-Litterman)

In [39]:
df

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights
Symbol,,,,,,
MMM,3M,Conglomerate,51260538679.09,1.09,0.10,0.08
AXP,American Express,Financial services,112503808568.25,1.02,0.10,0.01
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12
BA,Boeing,Aerospace and defense,118968728077.11,1.24,0.11,0.01
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18
CVX,Chevron,Petroleum industry,269853859026.24,0.82,0.09,0.01
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05
KO,Coca-Cola,Drink industry,245223815940.00,0.52,0.08,0.01


In [40]:
cov_matrix

,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DIS,...,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WBA,WMT
MMM,0.06,0.03,0.02,0.02,0.03,0.03,0.02,0.02,0.02,0.03,...,0.02,0.03,0.01,0.02,0.02,0.01,0.02,0.02,0.03,0.01
AXP,0.03,0.09,0.01,0.04,0.06,0.05,0.03,0.03,0.02,0.05,...,0.03,0.04,0.01,0.04,0.03,0.02,0.01,0.04,0.03,0.01
AMGN,0.02,0.01,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.01
AAPL,0.02,0.04,0.01,0.08,0.04,0.02,0.02,0.03,0.02,0.04,...,0.06,0.05,0.02,0.06,0.01,0.02,0.01,0.04,0.03,0.02
BA,0.03,0.06,0.01,0.04,0.15,0.05,0.04,0.03,0.01,0.06,...,0.03,0.05,0.01,0.05,0.03,0.01,0.01,0.04,0.04,0.01
CAT,0.03,0.05,0.01,0.02,0.05,0.09,0.05,0.02,0.01,0.04,...,0.02,0.03,0.01,0.02,0.03,0.01,0.01,0.03,0.03,0.01
CVX,0.02,0.03,0.01,0.02,0.04,0.05,0.08,0.02,0.01,0.03,...,0.01,0.02,0.00,0.02,0.03,0.01,0.01,0.02,0.03,0.01
CSCO,0.02,0.03,0.01,0.03,0.03,0.02,0.02,0.05,0.02,0.03,...,0.03,0.03,0.02,0.03,0.02,0.02,0.01,0.02,0.02,0.01
KO,0.02,0.02,0.01,0.02,0.01,0.01,0.01,0.02,0.03,0.02,...,0.02,0.02,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.01
DIS,0.03,0.05,0.01,0.04,0.06,0.04,0.03,0.03,0.02,0.10,...,0.04,0.05,0.01,0.05,0.02,0.01,0.01,0.04,0.03,0.01


In [41]:
# Compute the Market Cap weights 
# each Market cap / total Market cap = weigths proportion of that Market cap.

# e.g The Market cap weigt of `AAPL` is:
# 2899041777354.84 / (sum of all markets cap) = 0.25
df["mcap_weights"] = df.mcap.div(df.mcap.sum())
df

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights,mcap_weights
Symbol,,,,,,,
MMM,3M,Conglomerate,51260538679.09,1.09,0.10,0.08,0.00
AXP,American Express,Financial services,112503808568.25,1.02,0.10,0.01,0.01
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03,0.01
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12,0.25
BA,Boeing,Aerospace and defense,118968728077.11,1.24,0.11,0.01,0.01
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18,0.01
CVX,Chevron,Petroleum industry,269853859026.24,0.82,0.09,0.01,0.02
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05,0.02
KO,Coca-Cola,Drink industry,245223815940.00,0.52,0.08,0.01,0.02


In [42]:
# Expected return market
rm = 0.1

In [43]:
# normalization factor
nf = (rm - rfr) / (df.mcap_weights.T.dot(cov_matrix).dot(df.mcap_weights))
nf

1.4279777143522994

In [44]:
# implied returns (Black-Litterman) 
# This is the expected return based on Black-Litterman.

# cov_matrix * the market cap weigths * normalization factor + risk free return. 
df["BL_returns"] = cov_matrix.dot(df.mcap_weights) * nf + rfr
df

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights,mcap_weights,BL_returns
Symbol,,,,,,,,
MMM,3M,Conglomerate,51260538679.09,1.09,0.10,0.08,0.00,0.08
AXP,American Express,Financial services,112503808568.25,1.02,0.10,0.01,0.01,0.10
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03,0.01,0.07
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12,0.25,0.12
BA,Boeing,Aerospace and defense,118968728077.11,1.24,0.11,0.01,0.01,0.10
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18,0.01,0.08
CVX,Chevron,Petroleum industry,269853859026.24,0.82,0.09,0.01,0.02,0.08
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05,0.02,0.09
KO,Coca-Cola,Drink industry,245223815940.00,0.52,0.08,0.01,0.02,0.07


In [45]:
# CAPM Returns & BL Returns are implied returns (market expectations)
# We can take the avarage of two indicators - its a combination results of two indicators:
# CAPM return and Black-Litterman return. 
# it can be more accurate assumption on the market expected return.  
df["blended_returns"] = (df.CAPM_ret + df.BL_returns) / 2
df

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights,mcap_weights,BL_returns,blended_returns
Symbol,,,,,,,,,
MMM,3M,Conglomerate,51260538679.09,1.09,0.10,0.08,0.00,0.08,0.09
AXP,American Express,Financial services,112503808568.25,1.02,0.10,0.01,0.01,0.10,0.10
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03,0.01,0.07,0.08
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12,0.25,0.12,0.12
BA,Boeing,Aerospace and defense,118968728077.11,1.24,0.11,0.01,0.01,0.10,0.11
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18,0.01,0.08,0.10
CVX,Chevron,Petroleum industry,269853859026.24,0.82,0.09,0.01,0.02,0.08,0.08
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05,0.02,0.09,0.10
KO,Coca-Cola,Drink industry,245223815940.00,0.52,0.08,0.01,0.02,0.07,0.08


In [46]:
# investor opinions
df["returns_final"] = df["blended_returns"].copy()
df.loc["IBM", "returns_final"] = df.loc["IBM", "blended_returns"] + 0.03
df.loc["NKE", "returns_final"] = df.loc["NKE", "blended_returns"] + 0.02
df.loc["INTC", "returns_final"] = df.loc["INTC", "blended_returns"] - 0.03
df.loc["MCD", "returns_final"] = df.loc["MCD", "blended_returns"] - 0.02

In [47]:
df

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights,mcap_weights,BL_returns,blended_returns,returns_final
Symbol,,,,,,,,,,
MMM,3M,Conglomerate,51260538679.09,1.09,0.10,0.08,0.00,0.08,0.09,0.09
AXP,American Express,Financial services,112503808568.25,1.02,0.10,0.01,0.01,0.10,0.10,0.10
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03,0.01,0.07,0.08,0.08
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12,0.25,0.12,0.12,0.12
BA,Boeing,Aerospace and defense,118968728077.11,1.24,0.11,0.01,0.01,0.10,0.11,0.11
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18,0.01,0.08,0.10,0.10
CVX,Chevron,Petroleum industry,269853859026.24,0.82,0.09,0.01,0.02,0.08,0.08,0.08
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05,0.02,0.09,0.10,0.10
KO,Coca-Cola,Drink industry,245223815940.00,0.52,0.08,0.01,0.02,0.07,0.08,0.08


In [48]:
eweigths = np.full(noa, 1/noa)
eweigths

array([0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333])

In [49]:
bnds =  tuple((0.01, 0.25) for x in range(noa))
bnds

((0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25),
 (0.01, 0.25))

In [50]:
# Function to optimize (update - adjusted to calculate the optimal weigths with the returns_sinal.) 
def port_ret(weights):
    ret = df.returns_final.dot(weights)
    return ret

In [51]:
opts = sco.minimize(sharpe, # function to mizimize (for get the optimal sharp ratio)
                    eweigths, 
                    method = "SLSQP", 
                    bounds = bnds, 
                    constraints= cons)

c:\Users\97250\Desktop\sample_project_1\env\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [52]:
# See the weigths 
optimal_weights = opts["x"]
optimal_weights

array([0.01      , 0.01      , 0.01840317, 0.12934504, 0.01      ,
       0.05366591, 0.01      , 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.25      ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.01      ,
       0.06096964, 0.20104132, 0.01      , 0.01      , 0.01      ,
       0.01      , 0.01      , 0.01      , 0.01      , 0.05657492])

In [53]:
# Store the final weigths 
df["final_weights"] = pd.Series(data = optimal_weights, index = df.index)

In [54]:
df.sort_values(by = "final_weights", ascending = False)

,Company,Industry,mcap,beta,CAPM_ret,CAPM_weights,mcap_weights,BL_returns,blended_returns,returns_final,final_weights
Symbol,,,,,,,,,,,
IBM,IBM,Information technology,136072993704.79,0.44,0.07,0.01,0.01,0.08,0.07,0.10,0.25
NKE,Nike,Clothing industry,161489872726.66,1.36,0.12,0.03,0.01,0.10,0.11,0.13,0.20
AAPL,Apple,Information technology,2899041777354.84,1.28,0.11,0.12,0.25,0.12,0.12,0.12,0.13
MSFT,Microsoft,Information technology,2747483689486.73,1.04,0.10,0.01,0.24,0.12,0.11,0.11,0.06
WMT,Walmart,Retailing,447310352335.03,0.62,0.08,0.09,0.04,0.07,0.08,0.08,0.06
CAT,Caterpillar,Construction and mining,125533113286.48,1.45,0.12,0.18,0.01,0.08,0.10,0.10,0.05
AMGN,Amgen,Biopharmaceutical,143058425596.45,0.59,0.08,0.03,0.01,0.07,0.08,0.08,0.02
INTC,Intel,Semiconductor industry,163941789644.38,1.06,0.10,0.01,0.01,0.11,0.11,0.08,0.01
CSCO,Cisco,Information technology,214183813631.37,1.01,0.10,0.05,0.02,0.09,0.10,0.10,0.01
